In [8]:
from IPython import display
import os
def show_app(app, port = 9999, 
             width = 700, 
             height = 350, 
             offline = False,
            in_binder = None):
    in_binder ='JUPYTERHUB_SERVICE_PREFIX' in os.environ if in_binder is None else in_binder
    if in_binder:
        base_prefix = '{}proxy/{}/'.format(os.environ['JUPYTERHUB_SERVICE_PREFIX'], port)
        url = 'https://hub.mybinder.org{}'.format(base_prefix)
        app.config.requests_pathname_prefix = base_prefix
    else:
        url = 'http://localhost:%d' % port
        
    iframe = '<a href="{url}" target="_new">Open in new window</a><hr><iframe src="{url}" width={width} height={height}></iframe>'.format(url = url, 
                                                                                  width = width, 
                                                                                  height = height)
    
    display.display_html(iframe, raw = True)
    if offline:
        app.css.config.serve_locally = True
        app.scripts.config.serve_locally = True
    return app.run_server(debug=False, # needs to be false in Jupyter
                          host = '0.0.0.0',
                          port=port)

In [16]:
%run purchase_stock.ipynb

In [4]:
#import purchase_stock

ModuleNotFoundError: No module named 'purchase_stock'

In [ ]:
import robin_stocks as rs
import numpy as np
from datetime import datetime

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go

'''
with open('robinhood_config.txt', 'r') as f:
    file = f.read().split('\n')

f.close()
username = file[0]
password = file[1]
valid_code = file[2]
'''


username = 'arshyasharifian@gmail.com'
password = 'tyfpib-0xufna-hAxrod'
#valid_code = valid_code.split(' ')

login = rs.login(username, password)
min_time = datetime(datetime.today().year,
                    datetime.today().month,
                    datetime.today().day,
                    0, 0, 0)

max_time = datetime(datetime.today().year,
                    datetime.today().month,
                    datetime.today().day,
                    21, 0, 0)
p_list = []
t_list = []


#return a list of stocks



stocks_list = get_algo_stock_list()


def get_latest_buyPower():
    stocks_list = get_algo_stock_list()
    principle = float(rs.profiles.load_account_profile()['cash_available_for_withdrawal'])
    principle -= 25000
    #stock_price_list = rs.stocks.get_latest_price(stocks_list)
    if stocks_list:
        stock_price_list = rs.stocks.get_latest_price(inputSymbols=stocks_list)
        
    
        for stock in stock_price_list:
            principle += float(stock)

    return principle, datetime.now()


# Initiating a dash app
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.Div([
        html.H5('Live Graph Testing'),
    ], className='row'),
    html.Div([
        dcc.Graph(id='live_graph', animate=True),
        dcc.Interval(id='interval_component', interval=20*1000, n_intervals=0)  # interval = 20 sec
    ])
])


@app.callback(Output('live_graph', 'figure'),
              [Input('interval_component', 'n_intervals')])
def update_graph(n):
    global stocks_list
    global min_time
    global max_time
    global p_list
    global t_list

    principle, current_time = get_latest_buyPower()
    #principle += np.random.rand()

    p_list.append(principle)
    t_list.append(current_time)

    trace = go.Scatter(x=t_list,
                       y=p_list,
                       name='random plot',
                       mode='lines+markers')

    layout = go.Layout(xaxis=dict(range=[min_time, max_time])
                       )
    fig = {'data': [trace],
           'layout': layout
           }

    return fig


if __name__ == '__main__':
    app.run_server(debug=False)


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Jun/2020 12:10:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:10:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:10:28] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:10:28] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:10:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:10:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:10:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:11:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:11:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:11:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:11:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:11:46] "POST /_dash-update-component

127.0.0.1 - - [12/Jun/2020 12:24:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:24:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:25:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:25:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:25:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:25:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:25:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:25:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:26:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:26:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:26:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:26:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [12/Jun/2020 12:39:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:39:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:40:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:40:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:40:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:40:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:40:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:40:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:41:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:41:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:41:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:41:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [12/Jun/2020 12:54:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:54:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:55:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:55:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:55:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:55:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:55:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:55:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:56:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:56:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:56:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 12:56:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [12/Jun/2020 13:09:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 13:09:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 13:10:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 13:10:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 13:10:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 13:10:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 13:10:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 13:10:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 13:11:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 13:11:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 13:11:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 13:11:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [12/Jun/2020 13:24:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 13:24:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 13:25:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 13:25:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 13:25:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 13:25:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 13:25:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 13:25:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 13:26:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 13:26:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 13:26:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 13:26:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.